In [1]:
import rpy2.robjects as robjects

# Install package

In [2]:
robjects.r('''
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install(version = "3.16")
''')

R[write to console]: Bioconductor version 3.16 (BiocManager 1.30.19), R 4.2.1 (2022-06-23)

R[write to console]: Bioconductor version 3.16 (BiocManager 1.30.19), R 4.2.1 (2022-06-23)

R[write to console]: Old packages: 'bigmemory.sri', 'BiocParallel', 'learnr', 'rmarkdown'



Update all/some/none? [a/s/n]: n


<rpy2.robjects.vectors.StrVector object at 0x11e113880> [RTYPES.STRSXP]
R classes: ('character',)
[]

# Setup

In [2]:
robjects.r('''suppressWarnings(suppressMessages(require(netDx)))''')

1


# Data

In [3]:
robjects.r('''suppressMessages(library(curatedTCGAData))''')

'curatedT...,'MultiAss...,'Summariz...,...,'datasets','methods','base'


In [4]:
robjects.r('''
brca_data <- function(){
    brca <- suppressMessages(
    curatedTCGAData("BRCA",
               c("mRNAArray","miRNA*","Methylation_methyl27*"),
                dry.run=FALSE,version="1.1.38"))
    return(brca)
}
''')

<rpy2.robjects.functions.SignatureTranslatedFunction object at 0x12ab06c80> [RTYPES.CLOSXP]
R classes: ('function',)

In [5]:
brca = robjects.globalenv['brca_data']()

In [6]:
robjects.r('''
# setup brca data
prepareData <- function(dat, setBinary=FALSE) {
    staget <- sub("[abcd]","",sub("t","",colData(dat)$pathology_T_stage))
    staget <- suppressWarnings(as.integer(staget))
    colData(dat)$STAGE <- staget

    tmp <- colData(dat)$PAM50.mRNA
    if (!setBinary){
        idx <- which(tmp %in% c("Normal-like","HER2-enriched"))
    } else {
        idx <- union(which(tmp %in% c("Normal-like","HER2-enriched","Luminal B")),
                which(is.na(staget)))
    }
    idx <- union(idx, which(is.na(tmp)))
    pID <- colData(dat)$patientID
    tokeep <- setdiff(pID, pID[idx])
    dat <- dat[,tokeep,]
    pam50 <- colData(dat)$PAM50.mRNA

    
    smp <- sampleMap(dat)
    expr <- assays(dat)
    for (k in 1:length(expr)) {
        samps <- smp[which(smp$assay==names(expr)[k]),]
        notdup <- samps[which(!duplicated(samps$primary)),"colname"]
        #message(sprintf("%s: %i notdup", names(expr)[k], length(notdup)))
        dat[[k]] <- suppressMessages(dat[[k]][,notdup])
    }

    pID <- colData(dat)$patientID
    colData(dat)$ID <- pID
    colData(dat)$STATUS <- pam50
    colData(dat)$STATUS <- gsub(" ",".",colData(dat)$STATUS)
    colData(dat)$STATUS <- gsub("-",".",colData(dat)$STATUS)

    if (setBinary){
        st <- colData(dat)$STATUS
        st[which(!st %in% "Luminal.A")] <- "other"
        colData(dat)$STATUS <- st
    }

return(dat)
}
''')

<rpy2.robjects.functions.SignatureTranslatedFunction object at 0x12ab28400> [RTYPES.CLOSXP]
R classes: ('function',)

In [7]:
brca = robjects.globalenv['prepareData'](brca, True)

R[write to console]: harmonizing input:
  removing 28 sampleMap rows with 'colname' not in colnames of experiments

R[write to console]: harmonizing input:
  removing 44 sampleMap rows with 'colname' not in colnames of experiments

R[write to console]: harmonizing input:
  removing 19 sampleMap rows with 'colname' not in colnames of experiments



In [8]:
robjects.r('''
grouping_variables <- function(brca){
    expr <- assays(brca)
    groupList <- list()
    for (k in 1:length(expr)) { # loop over all layers
        cur <- expr[[k]]; nm <- names(expr)[k]

        # all measures from this layer go into our single PSN
        groupList[[nm]] <- list(nm=rownames(cur)) 

        # assign same layer name as in input data
        names(groupList[[nm]])[1] <- nm;
    
    }
    
    groupList[["clinical"]] <- list(
        age="patient.age_at_initial_pathologic_diagnosis",
          stage="STAGE"
    )
    
    tmp <- list(rownames(experiments(brca)[[1]]));
    names(tmp) <- names(brca)[1]
    groupList[[names(brca)[[1]]]] <- tmp

    tmp <- list(rownames(experiments(brca)[[2]]));
    names(tmp) <- names(brca)[2]
    groupList[[names(brca)[2]]] <- tmp

    tmp <- list(rownames(experiments(brca)[[3]]));
    names(tmp) <- names(brca)[3]
    groupList[[names(brca)[3]]] <- tmp
    
    
    return(groupList)
}
''')

<rpy2.robjects.functions.SignatureTranslatedFunction object at 0x12ab2bbc0> [RTYPES.CLOSXP]
R classes: ('function',)

In [9]:
groupList = robjects.globalenv['grouping_variables'](brca)

cnv_thresh = robjects.globalenv['cnv_thresh'](cnv_thresh)
mirna = robjects.globalenv['mirna'](mirna)
mrna = robjects.globalenv['mrna'](mrna)
proteins = robjects.globalenv['proteins'](proteins)

In [10]:
robjects.r('''
create_sims <- function(groupList){
    sims <- list(
      a="pearsonCorr",
      b="normDiff",
      c="pearsonCorr",
      d="pearsonCorr"
      )

    # map layer names to sims
    names(sims) <- names(groupList)
    return(sims)
}
''')

<rpy2.robjects.functions.SignatureTranslatedFunction object at 0x12ab53bc0> [RTYPES.CLOSXP]
R classes: ('function',)

In [11]:
sims = robjects.globalenv['create_sims'](groupList)

In [14]:
robjects.r('''
train_model <- function(brca, groupList, sims){
    nco <- round(parallel::detectCores()*0.75) # use 75% available cores
    message(sprintf("Using %i of %i cores", nco, parallel::detectCores()))
    
    outDir <- paste(tempdir(),"pred_output",sep=getFileSep()) # use absolute path
    if (file.exists(outDir)) unlink(outDir,recursive=TRUE)
    numSplits <- 2L

    t0 <- Sys.time()
    model <- suppressMessages(
    buildPredictor(
        dataList=brca,          ## your data
        groupList=groupList,    ## grouping strategy
        sims=sims,
        outDir=outDir,          ## output directory
        trainProp=0.8,          ## pct of samples to use to train model in
                                    ## each split
        numSplits=2L,            ## number of train/test splits
        featSelCutoff=1L,       ## threshold for calling something
                                    ## feature-selected
        featScoreMax=2L,    ## max score for feature selection
     numCores=nco,            ## set higher for parallelizing
     debugMode=FALSE,
     keepAllData=FALSE,     ## set to TRUE for debugging
     logging="none"     ## set to "default" for messages
      ))
    t1 <- Sys.time()
    print(t1-t0)
}
''')


<rpy2.robjects.functions.SignatureTranslatedFunction object at 0x12ab6de40> [RTYPES.CLOSXP]
R classes: ('function',)

In [15]:
model = robjects.globalenv['train_model'](brca, groupList, sims)

R[write to console]: Using 3 of 4 cores

R[write to console]: Errore in { : 
  task 1 failed - "memoria 'vector' esaurita (raggiunto il limite?)"



RRuntimeError: Errore in { : 
  task 1 failed - "memoria 'vector' esaurita (raggiunto il limite?)"
